# Slow Neural Style Transfer

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import tensorflow as tf
from VGGutils import *
from costutils import *

%matplotlib inline

In [ ]:
# Select which layers you want to contribute to the style loss function and their weighing
# Later layers = more "emergent" features
# Early layers = more emphasis on basic features (lines, basic shapes)

STYLE_LAYERS = [
    ('conv1_1', 0.01),
    ('conv2_1', 0.01),
    ('conv3_1', 0.02),
    ('conv4_1', 0.05),
    ('conv5_1', 0.1)]

In [ ]:
# Selection the activation layer
ACTIVATION_LAYER = 'conv4_2'

In [ ]:
# tf.InteractiveSession.close(sess)

In [ ]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()

In [ ]:
content_image = plt.imread("images/content/your_content_img.jpg")
content_image = reshape_and_normalize_image(content_image) # reshape just adds an extra dimension for VGGto be happy
imshow(content_image[0])

In [ ]:
style_image = plt.imread("images/style/your_style_img.jpg")
style_image = reshape_and_normalize_image(style_image)
imshow(style_image[0])

In [ ]:
"""
If desired, provide a binary mask to select regions where the style filter will be applied
"""

#content_mask = plt.imread("images/content/your_content_mask.jpg")
#content_mask = np.reshape(content_mask, ((1,) + content_mask.shape))
#content_mask = content_mask/255"""

In [ ]:
generated_image = generate_noise_image(content_image, 0.2)
imshow(generated_image[0])

In [ ]:
model = load_vgg_model(CONFIG.VGG_MODEL)

In [ ]:
# Assign the content image to be the input of the VGG model.  
sess.run(model['input'].assign(content_image))

# The activation layer output
out = model[ACTIVATION_LAYER]

# The content activation is the output from that layer evaluated
a_C = sess.run(out)

# Define the generated activation output, from the same layer
# We don't evaluate it yet
a_G = out

# Content cost
J_content = compute_content_cost(a_C, a_G)

In [ ]:
# Assign the style image to be the input of the VGG model. 
sess.run(model['input'].assign(style_image))

# Style cost
J_style = compute_style_cost(sess, model, STYLE_LAYERS, content_mask =  None)

In [ ]:
# Total cost
J = total_cost(J_content, J_style, alpha = 100, beta = 0.1)

In [ ]:
# Optimizer
# The idea is to minimize the total loss with respect to the generated image
# This means that the input image wile will be tweaked at each iteration
optimizer = tf.train.AdamOptimizer(2.0)
train_step = optimizer.minimize(J)

In [ ]:
def model_nst(sess, input_image, num_iterations = 1000):
    
    sess.run(tf.global_variables_initializer())
    
    # Set the noisy image as input to the model
    sess.run(model['input'].assign(input_image))
    
    for i in range(num_iterations):
    
        # Minimize the total cost
        sess.run(train_step)
        
        # Retrieves the generated image, which is the new "input" after minimizing the total cost
        generated_image = sess.run(model['input'])

        # Every 100 iterations, print the costs
        if i%100 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))
            
            # save current intermediate generated image to the output directory
            save_image("output/" + str(i) + ".png", generated_image)
    
    # save last generated image
    save_image('output/generated_image.jpg', generated_image)
    
    return generated_image

In [ ]:
model_nst(sess, generated_image)